## Treehouse Finance Problem Set

### Author: Sowmya Uppili Raghavan
### Date: 12th August 2021

### 1.0 Installing Dependencies

In [13]:
#!pip install yfinance (dependency for Yahoo Finance module)
#!pip install fix_yahoo_finance
#!pip install pandas_datareader

In [14]:
# data loading and manipulation
import pandas as pd
import numpy as np
from pandas_datareader import data as pdr

# Plotting 
import matplotlib.pyplot as plt
import seaborn
import matplotlib.mlab as mlab

# Statistical calculation
from scipy.stats import norm

# Data fetching from Yahoo Finance
import fix_yahoo_finance as yf

### 1.1. Answer

**Question**:

Give the name of employees, whose salaries are greater than their immediate manager’s.

**Answer:**

For this question, it requires the function to do the following set of steps:

1. Sort the table by manager_id and salary to identify the hierarchies across manager ids.
2. Fulfill Condition #1: Iterate through the rows of salary i.e. if the first row in the table has a lesser salary than the subsequent row;
3. Fulfill Condition #2: Iterate through the rows of manager_id i.e. if the first row in the table has a greater manager_id than the subsequent manager_id;
4. Append the value of the name for the index in which Conditions 1 & 2 are fulfilled.
5. Exceptions: If manager_id is undefined or 'NA' in this scenario, we have two possibilities for this: a) either the manager id has been wrongly entered within the system, in which case we exclude the data, or b) we assume that the individual is not a manager and therefore below in the hierarchy.

Using these conditions, the correct answer should be **Sally**, **Dan**, and assuming Phil with a manager_id = 'NULL' that he is a worker, **Phil**.

In [15]:
#read the data 
data = pd.read_csv('question1_sample.csv')
data = pd.DataFrame(data)

def get_names(data):
    data = data.sort_values(["manager_id", "salary"], ascending = False)
    lst = []
    for i in list(range(0, len(data)-2)):
        if data.iloc[i]['salary'] < data.iloc[i+1]['salary'] and (data.iloc[i]['manager_id'] > data.iloc[i+1]['manager_id']):
            lst.append(data.iloc[i+1]['name'])
    if data.iloc[len(data)-1]['salary'] > data.iloc[len(data)-2]['salary']:
        lst.append(data.iloc[len(data)-1]['name'])
    return lst, data

# Test Case #1: first three rows of data should return an empty list.
lst, test_1 = get_names(data.head(3))
print(test_1)
print(lst)

# Test Case #2: first five rows of data should return 'Sally'.
lst2, test_2 = get_names(data.head(5))
print(test_2)
print(lst2)

# Test Case #3: All rows of data should return 'Sally', 'Dan', 'Phil'.
lst3, test_3 = get_names(data)
print(test_3)
print(lst3)


   id   name  salary  manager_id
2   3  Sally     550         4.0
0   1   John     300         3.0
1   2   Mike     200         3.0
[]
   id   name  salary  manager_id
4   5    Joe     600         7.0
3   4   Jane     500         7.0
2   3  Sally     550         4.0
0   1   John     300         3.0
1   2   Mike     200         3.0
['Sally']
   id   name  salary  manager_id
4   5    Joe     600         7.0
3   4   Jane     500         7.0
2   3  Sally     550         4.0
5   6    Dan     600         3.0
0   1   John     300         3.0
1   2   Mike     200         3.0
6   7   Phil     550         NaN
['Sally', 'Dan', 'Phil']


## 1.2 Answer

**Question**: What is the average salary of employees who do not manage anyone? In the sample
above, that would be John, Mike, Joe and Dan, since they do not have anyone
reporting to them.

**Answer**: The process will work as follows:
1. Find the minimum manager_id value within the table.
2. Iterate through the rows of manager_id and if the manager_id is equal to the minimum value, add it to a sum variable.
3. If the row meets the previous condition, add it to the counter.



In [16]:
def get_average(data):
    lst2 = []
    sume = 0 
    counter = 0 
    for i in list(range(0, len(data)-1)):
        if data.iloc[i]['manager_id'] == data.manager_id.min():
            sume += data.iloc[i]['salary']
            counter += 1 
    print("The average salary of employees that do not manage anyone is: " + str(sume/counter))
    
get_average(data)

The average salary of employees that do not manage anyone is: 366.6666666666667


## 2.0 Answer

**Question**: Write a function ‘exists’ which takes a variable symbol v and returns whether v is defined.

**Answer**: There are multiple options for us to check if a variable symbol v is defined.

1. Option #1: Using a simple if, else pattern, check if v is in the global scope, else if false check if variable v is in the local scope. Finally, if both return False return "The variable is not defined."
2. Option #2: To check if a variable v exists without running it, using the 'try' statement in Python. 


In [222]:
## OPTION #1: Define if v exists in the local or global scope. 

def define(v):
    if v in globals():
        print("The variable exists and defined in a global scope.")
    elif v in locals():
        print("The variable exists and defined in a local scope.")
    else:
        print("The variable is not defined.")

        
## OPTION #2: 

try:
    v
except NameError:
    print("The variable is not defined.")
else:
    print("The variable is defined.")


The variable is not defined.


## 3.0 Answer

**Question**: Create a function to compute N layer of a Pascal Triangle.

**Answer**: Let n be the number of layers in the pascal triangle, where the  the sum of values in the nth row is 2n.
For i in the range of the n rows, print C where C = C * (i - j)/j, where j refers to i +1. 

In [218]:
def printPascal(n):
    for i in range(1, n + 1):
        C = 1; # used to represent C(i, j)
        for i in range(1, i + 1):
            print(C, end = " ")
            C = int(C * (i - j) / j)
        print("")
#test code 
n = 4;
printPascal(n);

1 
1 1 
1 2 1 
1 3 3 1 


## 4(a) Answer

**Question**: Using historical daily returns (Yahoo/Google Finance or any other market data
source), calculate VaR95% and CVaR95% of the portfolio as of 2016/12/31.

**Answer**: 

The **Historical value at risk (VaR)**, also known as historical simulation or the historical method, refers to a particular way of calculating VaR. In this approach we calculate VaR directly from past returns.

1. We pull data defined by the tickers and weights assigned in the portfolio for the 'Close' positions for each ticker.
2. Define start date as 2016/01/01 and end date as 2016/12/31 respectively.
3. In the get_historicVaR function, assign a historical_returns variable which returns the cumulative sum of all returns by weight for the time period.
3. **Historical VaR**: Return the 95th Percentile of these historical returns using the numpy percentile function.
3. **Conditional VaR**: Return the average value of  historical returns exceeding Historical VaR. 


In [60]:
# Create our portfolio of equities
tickers = ['AAPL','IBM', 'GOOG', 'BP', 'XOM', 'COST', 'GS']
weights = np.array([.15, .20, .20, .15, 0.10, 0.15, 0.05])

def get_data(tickers, weights):
    data = pdr.get_data_yahoo(tickers, start="2016-01-01", end='2016-12-31')['Close']
    returns = data.pct_change()
    returns = returns.dropna()
    return returns

def get_historicVaR(tickers,weights):
    returns = get_data(tickers,weights)
    historical_returns = (weights * returns.values).sum(axis=1)
    historic_VaR95 = np.percentile(historical_returns, 5)
    cvar_95 = historical_returns[historical_returns <= historic_VaR95].mean()
    print("Historic 95% VaR of Portfolio is:"+str(historic_VaR95))
    print("Historic 95% CVaR of Portfolio is:"+str(cvar_95))

get_historicVaR(tickers,weights)

Historic 95% VaR of Portfolio is:-0.014168767474557241
Historic 95% CVaR of Portfolio is:-0.021256080522287904


## 4(b) Answer 

**Question**: Using expected mean, covariance matrix and parametric method, calculate VaR95%
and CVaR95%

**Answer**:

VaR modeling determines the potential for loss in the entity being assessed and the probability of occurrence for the defined loss. Instead of relying on historical daily returns, the Parametric VaR method calculates the VaR based on a  probability density function which inputs the standard deviation, the mean and the confidence level of the portfolio. 

i. Calculating **Parametric Value at Risk (95%VaR)**:
1. Generate Var-Cov Matrix
2. Calculate Mean Returns ofr Each Stuck
3. Calculate mean Returns for Portfolio
3. Calculate Portfolio Std. Dev.
5. Determine Confidence Level - 0.95%

ii. The **Parametric Conditional Value at Risk (CVaR**, or expected shortfall (ES), asks what the average loss will be, conditional upon losses exceeding some threshold at a certain confidence level.


In [58]:
def get_parametricVar(tickers, weights, confidence_level=0.05):
    returns = get_data(tickers, weights)
    cov_matrix = returns.cov()
    avg_returns = returns.mean()
    port_mean = avg_returns@weights
    port_stdev = np.sqrt(weights.T @ cov_matrix @ weights)
    x = np.arange(-0.05, 0.05, 0.001)
    norm_dist = norm.pdf(x, port_mean, port_stdev)
    parametric_VaR95 = norm.ppf(confidence_level, port_mean, port_stdev)
    tail_loss = norm.expect(lambda x: x, loc = port_mean, scale = port_stdev, lb = parametric_VaR95)
    parametric_CVaR_95 = (1 / (1 - 0.95)) * tail_loss
    print("Parametric 95% VaR of Portfolio is: "+str(parametric_VaR95))
    print("Parametric 95% CVaR of Portfolio is: "+str(parametric_CVaR_95))


get_parametricVar(tickers, weights)

Parametric 95% VaR of Portfolio is:-0.01441002502874713
Parametric 95% CVaR of Portfolio is:0.02885832242185969


## 4(c) Answer 

**Answer**:
    
Understanding that there is a risk-return trade-off, a portfolio optimization strategy for each month would contain the following:

1. Using the covariance matrix above, it is apparent that the risk in the portfolio is not equivalent to the weighted average of individual stocks in the portfolio.
2. Instead, the risk is around volatitility, defined as the relationship of stock movement with each other. 
3. The Sharpe Ratio identifies the highest risk-free return across a permutation of all different weighted portfolios. Essentially, it identifies *i* in *n* number of portfolios with the highest return, *r*, given the lowest relative risk, *j*.
4. For each month, I would get the average returns and the covariance matrix as seen in the first helper function. 
5. Using the Scipy minimization function, I would calculate the max sharp_ratio for a portfolio in a given month using an adjusted monthly risk rate (defined as the monthly average of the risk rate provided by the U.S. treasury in the year 2015-2016). 
6. For that given month, I would assign

## Pseudo-Code

In [221]:
def get_monthly_returns(stock, ticker):
    data = pdr.get_data_yahoo(tickers, start="2016-01-01", end='2016-12-31')['Close']
    returns = data.pct_change()
    returns = returns.dropna()
    returns = returns.reset_index()
    returns['Date'] = pd.to_datetime(returns['Date'])
    returns['Month'] = returns['Date'].dt.month

def max_sharpe_ratio(stock, ticker, risk_free_rate=0.18):
    returns = get_monthly_returns(stock, ticker)
    cov_matrix = returns.cov()
    avg_returns = returns.mean()
    num_assets = len(avg_returns)
    args = (mean_returns, cov_matrix, risk_free_rate)
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1})
    bound = (0.0,1.0)
    bounds = tuple(bound for asset in range(num_assets))
    result = sco.minimize(neg_sharpe_ratio, num_assets*[1./num_assets,], args=args,
                        method='SLSQP', bounds=bounds, constraints=constraints)
    return result


## 5(a) Answer

**Question**: How many python files in the directory?

**Answer**: Using the git function on the command line, pattern match and aggregate the numnber of files with the ".py" suffix.

```
git ls-files | grep "\.py$" | wc -l
```

## 5(b) Answer

**Question**: How many lines of code in total, how many lines of comment line (empty line doesn’t
count)

**Answer**: Using svn, a custom command analogous to git, the 

```
!pip install svn
!svn ls -R https://github.com/“my-python-project”/branches/master | wc -l
```


## 5(c) Answer

**Question**: How many functions is defined in total?

**Answer**: The first step is to define the Github directory in your local path. The second step is to apply a function that locates the number of functions per file extension defined by the user.

```

import inspect
import importlib
import ast
import glob


files = glob.glob("/Users/Github/my-python-project" + '/**/*.py', recursive=True)

class CountFunc(ast.NodeVisitor):
    func_count = 0
    def visit_FunctionDef(self, node):
        self.func_count += 1


functions_count = []

for _file in files:
  p = ast.parse(open(_file).read())
  f = CountFunc()
  f.visit(p)
  functions_count.append(f.func_count)
  

print (f'Number of functions are {sum(functions_count)}')


```

## 5(d) Answer

**Question**: How many lines of changes from the current version against HEAD~3?

**Answer**: Using the git function on the command line, pattern match and aggregate the numnber of files with the ".py" suffix.

```
!git diff --stat HEAD-3 HEAD
```

## 6. Answer 

**Question**: In a text file, give me total number of appearance of “date” within the text file The date
format can appears in either one (or multiple) formats shown below:
    
**Answer**: Using regex pattern matching, match the following patterns:

1. DD/MM/YYYY: This has two digits with a max of 1 and 2 respectively, followed by two digits with a max of 3 and 1 for both digits respectively, and finally a final set of four digits with a / delimiter. 
2. MM/DD/YYYY: Similar ot the above pattern, this has two digits with a max of 3 and 1 followed by two digits with a max of 1 and 2, and a final set of four digits with a delimiter. 
3. DD/Jan/Feb/March/Dec/YYYY: Similar ot the above pattern, this has two digits with a max of 1 and 2 followed by four letters, and a final set of four digits with a delimiter. 

If any word in the text file matches the above patterns, add to a list and return the length of the list. 

In [220]:
from datetime import datetime

#f = open("demofile.txt", "r")

text = '10/2020/10, 10/10/2020, 2020/01/02, 01/Aug/2020'

def get_count(text):
    # Finds MM/DD/YYY and DD/MM/YYY
    pattern = "\d{2}[/-]\d{2}[/-]\d{4}"
    #Finds YYYY/MM/DD
    pattern2 = "\d{4}[/-]\d{2}[/-]\d{2}"
    ## Finds DD/Jan/Aug/Dec/etc../YYYY
    pattern3 = "\d{2}[/-][a-zA-Z]+[/-]\d{4}"
    #Create a pattern list
    pattern_lst = [pattern, pattern2, pattern3]
    found_regex_list = []
    for x in pattern_lst:
        if re.findall(x, text):
            some_list = re.findall(x, text)     
        for y in some_list:
            found_regex_list.append(y)
    print(len(found_regex_list))

get_count(text)

3
